In [32]:
import jieba
import pandas as pd
import re

In [2]:
with open("../Ch_trainfile_Sentiment_3000.txt","r",encoding='utf-8') as f:
    w = f.read()

In [4]:
a = [row.split('      ') for row in w.split('\n')]

In [6]:
df_comment = pd.DataFrame(a,columns=['scores','comment'])

In [9]:
df_comment.head()

,scores,comment
0,﻿0,商场几乎都没有活动
1,1,总体感觉蛮好的
2,-1,但是货品实在不咋滴
3,0,人比较少
4,0,还是贵的


# Cutting the words

In [33]:
#load stopwords set
stopwordset = set()
stopwordset.add(' ')
stopwordset.add('~')
stopwordset.add('、')
stopwordset.add('/')

# jieba custom setting
jieba.set_dictionary('./jieba_dict/dict.txt')

with open('./jieba_dict/stop_words.txt','r',encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))

for comment in df_comment.comment:
    seg_list = jieba.cut(comment, cut_all=False)
    #print('|'.join([word for word in seg_list if word not in stopwordset]))

In [12]:
# making a unique word lists

word_list = []
for comment in df_comment.comment:
    seg_list = jieba.cut(comment, cut_all=False)
    bb = [word for word in seg_list if word not in stopwordset]
    word_list.append(bb)

In [34]:
#word_list

## vectorize the word

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
word_cut = []
for each in word_list:
    word_cut.append(' '.join(each))

#將斷好詞的文章向量化
vectorizer = CountVectorizer()
vectorized_word = vectorizer.fit_transform(word_cut)

transformer = TfidfTransformer()
X_tfidf = transformer.fit_transform(vectorized_word)

In [35]:
#word_cut

In [16]:
# a sparse matrix
X_tfidf

<3001x2851 sparse matrix of type '<class 'numpy.float64'>'
	with 9670 stored elements in Compressed Sparse Row format>

In [18]:
## 評分 labels
train_y = df_comment.scores

In [19]:
len(train_y)

3001

# Trying classifier model

## SVM  using K-fold
### cross validation  - for all data 

In [20]:
from sklearn.model_selection import cross_val_predict
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(C=1, kernel="linear") # 可調參數
predicted = cross_val_predict(clf, X_tfidf, train_y, cv=5)
print("accuracy: {}".format(accuracy_score(train_y, predicted)))

C:\Users\hsuan\Anaconda3\envs\myproject\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


accuracy: 0.7187604131956015


## Rocchio - NearestCentroid

In [22]:
from sklearn.model_selection import cross_val_predict
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.metrics import accuracy_score

clf = NearestCentroid() # 可調參數
predicted = cross_val_predict(clf, X_tfidf, train_y, cv=5)
print("accuracy: {}".format(accuracy_score(train_y, predicted)))

accuracy: 0.6541152949016994


C:\Users\hsuan\Anaconda3\envs\myproject\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


## RandomForest


In [23]:
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf = RandomForestClassifier(n_estimators=300) # 可調參數
predicted = cross_val_predict(clf, X_tfidf, train_y, cv=5)
print("accuracy: {}".format(accuracy_score(train_y, predicted)))

C:\Users\hsuan\Anaconda3\envs\myproject\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


accuracy: 0.716427857380873


# split up the dataset
 with classifier

In [24]:
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, train_y, test_size = 0.3, random_state = 0)

### Rocchio

In [26]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
clf = NearestCentroid().fit(X_train,y_train)
y_pred = clf.predict(X_test) # predict : testing data
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.65815760266370704

### KNN

In [25]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)
accuracy_score(y_test, neigh.predict(X_test))

0.60044395116537186

### SVM

In [28]:
from sklearn.svm import SVC

clf1 = SVC(C=1000.0, kernel='rbf')
clf1.fit(X_train, y_train)
accuracy_score(y_test, clf1.predict(X_test))

0.71587125416204223

The C parameter tells the SVM optimization how much you want to avoid misclassifying each training example. For large values of C, the optimization will choose a smaller-margin hyperplane if that hyperplane does a better job of getting all the training points classified correctly. Conversely, a very small value of C will cause the optimizer to look for a larger-margin separating hyperplane, even if that hyperplane misclassifies more points. For very tiny values of C, you should get misclassified examples, often even if your training data is linearly separable.

### Bayes

In [29]:
from sklearn.naive_bayes import MultinomialNB
mgnb = MultinomialNB()
mgnb.fit(X_train, y_train)
accuracy_score(y_test, mgnb.predict(X_test))

0.64594894561598226

# New data testing

In [30]:
doc_news = ['生意 红火']
X_new_count = vectorized_word = vectorizer.transform(doc_news)
X_new_tfidf = transformer.transform(X_new_count)
predict = clf.predict(X_new_tfidf)

In [31]:
predict

array(['0'], dtype=object)